In [1]:
# Run merge_order_and_sort_by_date before this file
import numpy as np
import pandas as pd
import pickle
import gzip
import gc
import random
import csv
import os
from datetime import datetime
from torch.utils.data.dataset import random_split

In [2]:
DATASET_NAME = "TaFeng"
# DATASET_NAME = "Dunnhumby"

In [3]:
df_history = pd.read_csv(f"./data/raw_data/{DATASET_NAME}_history.csv")
df_history

,CUSTOMER_ID,ORDER_NUMBER,MATERIAL_NUMBER
0,1069,1,0
1,1069,1,1
2,1113,1,10
3,1113,1,11
4,1113,2,10
...,...,...,...
290715,20002000,23,9696
290716,20002000,23,3243
290717,20002000,23,6726
290718,20002000,23,7995


In [4]:
df_future = pd.read_csv(f"./data/raw_data/{DATASET_NAME}_future.csv")
df_future

,CUSTOMER_ID,ORDER_NUMBER,MATERIAL_NUMBER
0,1069,1,2
1,1069,1,3
2,1069,1,4
3,1069,2,5
4,1069,2,1
...,...,...,...
276289,20002000,1,1379
276290,20002000,1,7079
276291,20002000,1,2660
276292,20002000,2,309


In [5]:
# 取的所有用戶的所有購物籃項目id(輸出是一個三維串列)、用戶ID、購物籃項目id、購物籃
def get_users_history_cartitemid(df_group):
    
    get_cart_itemid = lambda cart: [df_history["MATERIAL_NUMBER"][index] for index in cart]
    
    last_user_id = 0
    user_cart_itemid_list = []
    cart_itemid_list = []
    cart_size_list = []
    for index, (userid, cartid) in enumerate(df_group.groups):
        if last_user_id == 0 or last_user_id == userid:
            item_list = list(df_group.groups[(userid, cartid)])
            cart_itemid_list.append(get_cart_itemid(item_list))
            cart_size_list.append(len(item_list))
        else:
            user_cart_itemid_list.append((last_user_id, cart_itemid_list, cart_size_list))
            cart_itemid_list = []
            cart_size_list = []
            item_list = list(df_group.groups[(userid, cartid)])
            cart_itemid_list.append(get_cart_itemid(item_list))
            cart_size_list.append(len(item_list))
        last_user_id = userid
    user_cart_itemid_list.append((last_user_id, cart_itemid_list, cart_size_list))
    return user_cart_itemid_list

In [6]:
# 取的所有用戶的所有購物籃項目id(輸出是一個三維串列)、用戶ID、購物籃項目id、購物籃
def get_users_future_cartitemid(df_group):
    
    get_cart_itemid = lambda cart: [df_future["MATERIAL_NUMBER"][index] for index in cart]
    
    last_user_id = 0
    user_cart_itemid_list = []
    cart_itemid_list = []
    cart_size_list = []
    for index, (userid, cartid) in enumerate(df_group.groups):
        if last_user_id == 0 or last_user_id == userid:
            item_list = list(df_group.groups[(userid, cartid)])
            cart_itemid_list.append(get_cart_itemid(item_list))
            cart_size_list.append(len(item_list))
        else:
            user_cart_itemid_list.append((last_user_id, cart_itemid_list, cart_size_list))
            cart_itemid_list = []
            cart_size_list = []
            item_list = list(df_group.groups[(userid, cartid)])
            cart_itemid_list.append(get_cart_itemid(item_list))
            cart_size_list.append(len(item_list))
        last_user_id = userid
    user_cart_itemid_list.append((last_user_id, cart_itemid_list, cart_size_list))
    return user_cart_itemid_list

In [7]:
history_group = df_history.groupby(["CUSTOMER_ID", "ORDER_NUMBER"])
history_cart_itemid_list = get_users_history_cartitemid(history_group)
print("history:", history_cart_itemid_list[:2])
future_group = df_future.groupby(["CUSTOMER_ID", "ORDER_NUMBER"])
future_cart_itemid_list = get_users_future_cartitemid(future_group)
print("future:", future_cart_itemid_list[:1])

history: [(1069, [[0, 1]], [2]), (1113, [[10, 11], [10, 12, 13], [14, 15, 16, 17, 18, 19]], [2, 3, 6])]
future: [(1069, [[2, 3, 4], [5, 1, 6, 7, 8], [9]], [3, 5, 1])]


In [8]:
history_file = f"./data/{DATASET_NAME}_history.csv" 
future_file = f"./data/{DATASET_NAME}_future.csv"
headers = ['CUSTOMER_ID','ORDER_NUMBER','MATERIAL_NUMBER']
print('Preprocessing...')
user_list = list()
with open(future_file, 'w', newline="") as csvfile:
    user_list = list()
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    for i in range(len(future_cart_itemid_list)):
        uid = future_cart_itemid_list[i][0]
        user_list.append(uid)
        
        date = 1
        for basket in future_cart_itemid_list[i][1]:
            for item in basket:
                row = []
                row.append(uid)
                row.append(date)
                row.append(item)
                writer.writerow(row)
            date += 1
            

with open(history_file, 'w', newline="") as csvfile:
    user_list = list()
    writer = csv.writer(csvfile)
    writer.writerow(headers)
    for i in range(len(history_cart_itemid_list)):
        uid = history_cart_itemid_list[i][0]
        user_list.append(uid)
        
        date = 1
        for basket in history_cart_itemid_list[i][1]:
            for item in basket:
                row = []
                row.append(uid)
                row.append(date)
                row.append(item)
                writer.writerow(row)
            date += 1            
print('Done!')

Preprocessing...
Done!
